In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
plt.style.use(['bmh'])
from sklearn.model_selection import RepeatedKFold

from sklearn.model_selection import train_test_split
from sklearn.utils import resample
from sklearn.model_selection import KFold
from sklearn.model_selection import LeaveOneOut
from sklearn.metrics import roc_auc_score
from sklearn.metrics import brier_score_loss
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import precision_recall_fscore_support as score
from sklearn.metrics import roc_curve, roc_auc_score, make_scorer
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.linear_model import SGDClassifier, LogisticRegressionCV,LogisticRegression
from sklearn.feature_selection import RFE, RFECV
from sklearn.linear_model import RidgeClassifierCV, LassoCV, LassoLarsCV, LassoLarsIC, Ridge, Lasso
from sklearn import svm, datasets
from scipy.stats import boxcox
from sklearn.svm import SVC
from sklearn.cluster import KMeans
from sklearn.neighbors import KNeighborsClassifier
from numpy import mean
from numpy import std
from sklearn.datasets import make_classification
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import RepeatedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
from itertools import product
from sklearn.linear_model import LassoLarsCV
from sklearn.datasets import make_regression
from sklearn.metrics import mean_absolute_error,r2_score
from mlxtend.plotting import plot_sequential_feature_selection as plot_sfs
from sklearn.linear_model import ElasticNet
import os
from pandas import ExcelWriter
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.cluster import KMeans
from sklearn.metrics import adjusted_rand_score
from sklearn.cluster import DBSCAN
from sklearn.metrics import silhouette_score, adjusted_rand_score
from kneed import KneeLocator

In [5]:
%%javascript
(function(on) {
const e=$( "<a>Setup failed</a>" );
const ns="js_jupyter_suppress_warnings";
var cssrules=$("#"+ns);
if(!cssrules.length) cssrules = $("<style id='"+ns+"' type='text/css'>div.output_stderr { } </style>").appendTo("head");
e.click(function() {
    var s='Showing';  
    cssrules.empty()
    if(on) {
        s='Hiding';
        cssrules.append("div.output_stderr, div[data-mime-type*='.stderr'] { display:none; }");
    }
    e.text(s+' warnings (click to toggle)');
    on=!on;
}).click();
$(element).append(e);
})(true);

<IPython.core.display.Javascript object>

# Comparing to random

In [8]:
def Compare_random(data,cv,NumofR):
    #Comparing to random
    #senario1
    x_column_list=data.drop(columns=['Link_ID', 'y_binary', 'Yield_per_plant']).columns
    data_train,data_val=train_test_split(data,train_size=0.8, random_state=42)
    
    pipeRF = Pipeline([('classifier', [RandomForestClassifier()])])
    param_grid = [
    {'classifier' : [RandomForestClassifier()],
    'classifier__criterion':('gini','entropy'),
    'classifier__class_weight':('balanced','auto')}]
    clf = GridSearchCV(pipeRF, param_grid = param_grid, cv = cv, n_jobs=-1, scoring='f1_weighted')

    clf.fit(data_train[x_column_list],data_train['y_binary'])
    best_clf=clf.best_estimator_
    y_valid=best_clf.predict(data_val[x_column_list])
    
    report_RF = classification_report(data_val['y_binary'],y_valid,output_dict=True) 
    
    F1_0_class_RF=report_RF['0.0']['f1-score']
    F1_1_class_RF=report_RF['1.0']['f1-score']
    
    pipeRF = Pipeline([('classifier', [RandomForestClassifier()])])
    param_grid = [
    {'classifier' : [RandomForestClassifier()],
    'classifier__criterion':('gini','entropy'),
    'classifier__class_weight':('balanced','auto')}]
    totalF1_0=[]
    totalF1_1=[]
    for i in range(1,NumofR):
        RD1= np.random.rand(data[x_column_list].shape[0],data[x_column_list].shape[1])
        RD1=pd.DataFrame(RD1)
        RD1.columns =data[x_column_list].columns
        RD11=RD1.div(RD1.sum(axis=1), axis=0)        
        clf = GridSearchCV(pipeRF, param_grid = param_grid, cv = cv, n_jobs=-1, scoring='f1_weighted')
        clf.fit(RD11.iloc[0:data_train.shape[0],0:len(x_column_list)], data_train['y_binary'])
        best_clf=clf.best_estimator_
        y_valid=best_clf.predict(RD11.iloc[data_train.shape[0]:,0:len(x_column_list)])
        report_Random1 = classification_report(data_val['y_binary'],y_valid,output_dict=True)
        totalF1_0.append(report_Random1 ['0.0']['f1-score'])
        totalF1_1.append(report_Random1 ['1.0']['f1-score'])    
    r1=pd.DataFrame()
    r1['0']= totalF1_0
    r1['1']= totalF1_1
    EV1=r1[(r1['0'] >= F1_0_class_RF) & (r1['1'] >=F1_1_class_RF)].shape[0]/ NumofR
    #senario2
    totalF1_0_2=[]
    totalF1_1_2=[]
    for i in range(1,NumofR):
        RD2=pd.DataFrame(np.random.rand(data[x_column_list].shape[0],data[x_column_list].shape[1]))
        RD2.columns=data[x_column_list].columns
        for col in data[x_column_list].columns:
            dd=np.where(data[x_column_list][col]==0)[0]
            if len(dd)>0:
                RD2[col][dd]=0
        RD22=RD2.div(RD2.sum(axis=1), axis=0)
        clf = GridSearchCV(pipeRF, param_grid = param_grid, cv = cv, n_jobs=-1, scoring='f1_weighted')
        clf.fit(RD22[x_column_list][0:data_train.shape[0]], data_train['y_binary'])
        best_clf=clf.best_estimator_
        y_valid=best_clf.predict(RD22[x_column_list][data_train.shape[0]:])
        report_Random2 = classification_report(data_val['y_binary'],y_valid,output_dict=True)
        totalF1_0_2.append(report_Random2['0.0']['f1-score'])
        totalF1_1_2.append(report_Random2['1.0']['f1-score'])
    r2=pd.DataFrame()
    r2['0']= totalF1_0_2
    r2['1']= totalF1_1_2
    EV2=r2[(r2['0'] >= F1_0_class_RF) & (r2['1'] >=F1_1_class_RF)].shape[0]/ NumofR
    #senario3
    totalF1_0_3=[]
    totalF1_1_3=[]
    for i in range(1,NumofR):
        YR=np.random.permutation(data['y_binary'].values)
        clf = GridSearchCV(pipeRF, param_grid = param_grid, cv = cv, n_jobs=-1, scoring='f1_weighted')
        clf.fit(data_train[x_column_list],  YR[0:data_train[x_column_list].shape[0]])
        best_clf=clf.best_estimator_
        y_valid=best_clf.predict(data_val[x_column_list])
        report_Random3 = classification_report(data_val['y_binary'],y_valid,output_dict=True)
        totalF1_0_3.append(report_Random3['0.0']['f1-score'])
        totalF1_1_3.append(report_Random3['1.0']['f1-score'])
    r3=pd.DataFrame()
    r3['0']= totalF1_0_3
    r3['1']= totalF1_1_3
    EV3=r3[(r3['0'] >= F1_0_class_RF) & (r3['1'] >=F1_1_class_RF)].shape[0]/ NumofR
    #Senario4
    totalF1_0_4=[]
    totalF1_1_4=[]
    for i in range(1,NumofR):
        DR=data[x_column_list].sample(frac=1)
        clf = GridSearchCV(pipeRF, param_grid = param_grid, cv = cv, n_jobs=-1, scoring='f1_weighted')
        clf.fit(DR[0:data_train[x_column_list].shape[0]],  data_train['y_binary'])
        best_clf=clf.best_estimator_
        y_valid=best_clf.predict(DR[data_train[x_column_list].shape[0]:])
        report_Random4 = classification_report(data_val['y_binary'],y_valid,output_dict=True)
        totalF1_0_4.append(report_Random4['0.0']['f1-score'])
        totalF1_1_4.append(report_Random4['1.0']['f1-score'])
    r4=pd.DataFrame()
    r4['0']= totalF1_0_4
    r4['1']= totalF1_1_4
    EV4=r4[(r4['0'] >= F1_0_class_RF) & (r4['1'] >=F1_1_class_RF)].shape[0]/ NumofR
    ev_value=[EV1,EV2,EV3,EV4]
    return r1,r2,r3,r4,ev_value,F1_0_class_RF,F1_1_class_RF


In [9]:
df1=pd.read_csv("./Main Data Files/binary-data.csv")
df2 = pd.read_csv('./Main Data Files/metadata.csv')
df1=df1.drop('Variety2',axis=1)
df1.rename(columns={'Yield_per_plant':"y_binary"}, inplace=True)
df2 = df2[['Unnamed: 0', 'Yield_per_plant']]
df2.rename(columns={'Unnamed: 0':'Link_ID'}, inplace=True)
data_original = pd.merge(df1, df2, on='Link_ID')

data=pd.read_csv("/Users/rosa/Desktop/ALLWork/Madison/Project/Soil-nn/DATA/Netcomi_data/ 1 _ 1 .csv")
#data=pd.read_csv("/Users/rosa/Desktop/ALLWork/Madison/Project/Soil-nn/DATA/Netcomi_data/ 1 _ 2 .csv")
#data=pd.read_csv("/Users/rosa/Desktop/ALLWork/Madison/Project/Soil-nn/DATA/Netcomi_data/ 1 _ 1 .csv")
#data=pd.read_csv("/Users/rosa/Desktop/ALLWork/Madison/Project/Soil-nn/DATA/Netcomi_data/ 3 _ 3 .csv")


data.rename(columns={'Unnamed: 0':'Link_ID'}, inplace=True)
data['y_binary']=data_original['y_binary'].values
data['Yield_per_plant']=data_original['Yield_per_plant'].values


cv=RepeatedKFold(n_splits=10,n_repeats=3, random_state=100)
NumofR=100
out_random=Compare_random(data,cv,NumofR)

/Users/rosa/opt/anaconda3/envs/testEnv/lib/python3.10/site-packages/sklearn/model_selection/_validation.py:378: FitFailedWarning: 
60 fits failed out of a total of 120.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
60 fits failed with the following error:
Traceback (most recent call last):
  File "/Users/rosa/opt/anaconda3/envs/testEnv/lib/python3.10/site-packages/sklearn/model_selection/_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Users/rosa/opt/anaconda3/envs/testEnv/lib/python3.10/site-packages/sklearn/pipeline.py", line 382, in fit
    self._final_estimator.fit(Xt, y, **fit_params_last_step)
  File "/Users/rosa/opt/anaconda3/envs/testEnv/lib/python3.10/site-packages/sklearn

/Users/rosa/opt/anaconda3/envs/testEnv/lib/python3.10/site-packages/sklearn/model_selection/_validation.py:378: FitFailedWarning: 
60 fits failed out of a total of 120.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
60 fits failed with the following error:
Traceback (most recent call last):
  File "/Users/rosa/opt/anaconda3/envs/testEnv/lib/python3.10/site-packages/sklearn/model_selection/_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Users/rosa/opt/anaconda3/envs/testEnv/lib/python3.10/site-packages/sklearn/pipeline.py", line 382, in fit
    self._final_estimator.fit(Xt, y, **fit_params_last_step)
  File "/Users/rosa/opt/anaconda3/envs/testEnv/lib/python3.10/site-packages/sklearn

/Users/rosa/opt/anaconda3/envs/testEnv/lib/python3.10/site-packages/sklearn/model_selection/_validation.py:378: FitFailedWarning: 
60 fits failed out of a total of 120.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
60 fits failed with the following error:
Traceback (most recent call last):
  File "/Users/rosa/opt/anaconda3/envs/testEnv/lib/python3.10/site-packages/sklearn/model_selection/_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Users/rosa/opt/anaconda3/envs/testEnv/lib/python3.10/site-packages/sklearn/pipeline.py", line 382, in fit
    self._final_estimator.fit(Xt, y, **fit_params_last_step)
  File "/Users/rosa/opt/anaconda3/envs/testEnv/lib/python3.10/site-packages/sklearn

/Users/rosa/opt/anaconda3/envs/testEnv/lib/python3.10/site-packages/sklearn/model_selection/_validation.py:378: FitFailedWarning: 
60 fits failed out of a total of 120.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
60 fits failed with the following error:
Traceback (most recent call last):
  File "/Users/rosa/opt/anaconda3/envs/testEnv/lib/python3.10/site-packages/sklearn/model_selection/_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Users/rosa/opt/anaconda3/envs/testEnv/lib/python3.10/site-packages/sklearn/pipeline.py", line 382, in fit
    self._final_estimator.fit(Xt, y, **fit_params_last_step)
  File "/Users/rosa/opt/anaconda3/envs/testEnv/lib/python3.10/site-packages/sklearn

/Users/rosa/opt/anaconda3/envs/testEnv/lib/python3.10/site-packages/sklearn/model_selection/_validation.py:378: FitFailedWarning: 
60 fits failed out of a total of 120.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
60 fits failed with the following error:
Traceback (most recent call last):
  File "/Users/rosa/opt/anaconda3/envs/testEnv/lib/python3.10/site-packages/sklearn/model_selection/_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Users/rosa/opt/anaconda3/envs/testEnv/lib/python3.10/site-packages/sklearn/pipeline.py", line 382, in fit
    self._final_estimator.fit(Xt, y, **fit_params_last_step)
  File "/Users/rosa/opt/anaconda3/envs/testEnv/lib/python3.10/site-packages/sklearn

...

/Users/rosa/opt/anaconda3/envs/testEnv/lib/python3.10/site-packages/sklearn/model_selection/_validation.py:378: FitFailedWarning: 
60 fits failed out of a total of 120.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
60 fits failed with the following error:
Traceback (most recent call last):
  File "/Users/rosa/opt/anaconda3/envs/testEnv/lib/python3.10/site-packages/sklearn/model_selection/_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Users/rosa/opt/anaconda3/envs/testEnv/lib/python3.10/site-packages/sklearn/pipeline.py", line 382, in fit
    self._final_estimator.fit(Xt, y, **fit_params_last_step)
  File "/Users/rosa/opt/anaconda3/envs/testEnv/lib/python3.10/site-packages/sklearn

/Users/rosa/opt/anaconda3/envs/testEnv/lib/python3.10/site-packages/sklearn/model_selection/_validation.py:378: FitFailedWarning: 
60 fits failed out of a total of 120.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
60 fits failed with the following error:
Traceback (most recent call last):
  File "/Users/rosa/opt/anaconda3/envs/testEnv/lib/python3.10/site-packages/sklearn/model_selection/_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Users/rosa/opt/anaconda3/envs/testEnv/lib/python3.10/site-packages/sklearn/pipeline.py", line 382, in fit
    self._final_estimator.fit(Xt, y, **fit_params_last_step)
  File "/Users/rosa/opt/anaconda3/envs/testEnv/lib/python3.10/site-packages/sklearn

/Users/rosa/opt/anaconda3/envs/testEnv/lib/python3.10/site-packages/sklearn/model_selection/_validation.py:378: FitFailedWarning: 
60 fits failed out of a total of 120.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
60 fits failed with the following error:
Traceback (most recent call last):
  File "/Users/rosa/opt/anaconda3/envs/testEnv/lib/python3.10/site-packages/sklearn/model_selection/_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Users/rosa/opt/anaconda3/envs/testEnv/lib/python3.10/site-packages/sklearn/pipeline.py", line 382, in fit
    self._final_estimator.fit(Xt, y, **fit_params_last_step)
  File "/Users/rosa/opt/anaconda3/envs/testEnv/lib/python3.10/site-packages/sklearn

/Users/rosa/opt/anaconda3/envs/testEnv/lib/python3.10/site-packages/sklearn/model_selection/_validation.py:378: FitFailedWarning: 
60 fits failed out of a total of 120.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
60 fits failed with the following error:
Traceback (most recent call last):
  File "/Users/rosa/opt/anaconda3/envs/testEnv/lib/python3.10/site-packages/sklearn/model_selection/_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Users/rosa/opt/anaconda3/envs/testEnv/lib/python3.10/site-packages/sklearn/pipeline.py", line 382, in fit
    self._final_estimator.fit(Xt, y, **fit_params_last_step)
  File "/Users/rosa/opt/anaconda3/envs/testEnv/lib/python3.10/site-packages/sklearn

/Users/rosa/opt/anaconda3/envs/testEnv/lib/python3.10/site-packages/sklearn/model_selection/_validation.py:378: FitFailedWarning: 
60 fits failed out of a total of 120.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
60 fits failed with the following error:
Traceback (most recent call last):
  File "/Users/rosa/opt/anaconda3/envs/testEnv/lib/python3.10/site-packages/sklearn/model_selection/_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Users/rosa/opt/anaconda3/envs/testEnv/lib/python3.10/site-packages/sklearn/pipeline.py", line 382, in fit
    self._final_estimator.fit(Xt, y, **fit_params_last_step)
  File "/Users/rosa/opt/anaconda3/envs/testEnv/lib/python3.10/site-packages/sklearn

In [ ]:
print(out_random[4])

In [ ]:
plt.figure(figsize=(15,25))
plt.subplot(4,1,1)
r1=out_random[0]
plt.scatter(r1['0'],r1['1'])
plt.scatter(out_random[5],out_random[6])
plt.subplot(4,1,2)
r2=out_random[1]
plt.scatter(r2['0'],r2['1'])
plt.scatter(out_random[5],out_random[6])
plt.subplot(4,1,3)
r3=out_random[2]
plt.scatter(r3['0'],r3['1'])
plt.scatter(out_random[5],out_random[6])
plt.subplot(4,1,4)
r4=out_random[3]
plt.scatter(r4['0'],r4['1'])
plt.scatter(out_random[5],out_random[6])
plt.savefig('/Users/rosa/Desktop/ALLWork/Madison/Project/Soil-nn/Report/Random/compareTorandom1-1.png',facecolor='white')

In [ ]:
sns.boxplot(x="variable", y="value", data=pd.melt(r1))

In [ ]:
sns.boxplot(x="variable", y="value", data=pd.melt(r2))

In [ ]:
sns.boxplot(x="variable", y="value", data=pd.melt(r3))

In [ ]:
sns.boxplot(x="variable", y="value", data=pd.melt(r4))